In [ ]:
# Run this once per fresh runtime / virtual‑env
!pip install --quiet \
    openml \
    scikit-learn \
    pandas \
    numpy \
    pytorch-tabnet \
    torch \
    tensorflow

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.4/160.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.3 MB/s eta 0:00:00
   ━

In [ ]:
#   Install the missing packages
!pip install --quiet openml scikit-learn pandas numpy pytorch-tabnet torch tensorflow

#   (Optional) Verify that openml is now importable
import importlib, pkg_resources, sys
for p in ["openml", "sklearn", "pytorch_tabnet"]:
    print(p, "→", importlib.import_module(p).__version__)

<ipython-input-3-45b62c9fadbe>:5: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import importlib, pkg_resources, sys


openml → 0.15.1
sklearn → 1.6.1


AttributeError: module 'pytorch_tabnet' has no attribute '__version__'

In [ ]:
import numpy as np
import pandas as pd

from scipy.stats import skew, kurtosis, entropy
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Reshape, Conv1D, GlobalMaxPooling1D,
    SimpleRNN, LSTM, Dropout, BatchNormalization, Dense
)
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

from lightgbm import LGBMRanker

# Print TensorFlow version for debugging
print("TensorFlow version:", tf.__version__)

# Reset Keras session and set float type
K.clear_session()
K.set_floatx('float32')

# ──────────────────────────────────────────────────────────────────────────────
# 1) SYNTHETIC DATASETS: GENERATE 40, SPLIT 30/10
# ──────────────────────────────────────────────────────────────────────────────
rng = np.random.RandomState(0)
datasets = {}
for i in range(40):
    n_samples  = rng.randint(200, 2000)
    n_features = rng.randint(5, 50)
    n_classes  = rng.randint(2, 6)
    X, y = make_classification(
        n_samples=n_samples,
        n_features=n_features,
        n_informative=int(n_features*0.6),
        n_redundant=int(n_features*0.2),
        n_repeated=0,
        n_classes=n_classes,
        n_clusters_per_class=1,     # avoid the 2^n_informative check
        flip_y=0.03,
        class_sep=rng.uniform(0.5,2.0),
        random_state=i
    )
    # force purely numeric
    datasets[f"syn_{i}"] = (X.astype(np.float32), y.astype(np.int32))

all_keys = list(datasets.keys())
train_keys, test_keys = train_test_split(all_keys, train_size=30, random_state=1)
train_ds = {k: datasets[k] for k in train_keys}
test_ds  = {k: datasets[k] for k in test_keys}

# ──────────────────────────────────────────────────────────────────────────────
# 2) MODEL BUILDERS
# ──────────────────────────────────────────────────────────────────────────────
def build_cnn(d, c):
    m = Sequential(name="CNN1D")
    m.add(Reshape((d,1), input_shape=(d,), dtype='float32'))
    m.add(Conv1D(64, 3, activation="relu"))
    m.add(BatchNormalization())
    m.add(GlobalMaxPooling1D())
    m.add(Dense(64, activation="relu"))
    m.add(Dense(c, activation="softmax"))
    m.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return m

def build_rnn(d, c):
    m = Sequential(name="RNN1D")
    m.add(Reshape((d,1), input_shape=(d,), dtype='float32'))
    m.add(SimpleRNN(64))
    m.add(Dropout(0.3))
    m.add(Dense(c, activation="softmax"))
    m.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return m

def build_lstm(d, c):
    m = Sequential(name="LSTM1D")
    m.add(Reshape((d,1), input_shape=(d,), dtype='float32'))
    m.add(LSTM(64))
    m.add(Dropout(0.3))
    m.add(Dense(c, activation="softmax"))
    m.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return m

# ──────────────────────────────────────────────────────────────────────────────
# 3) BENCHMARK FUNCTION
# ──────────────────────────────────────────────────────────────────────────────
def benchmark(ds_dict):
    rows = []
    for idx, (name, (X, y)) in enumerate(ds_dict.items(), 1):
        print(f"\n→ [{idx}/{len(ds_dict)}] {name}")
        n_i, n_f = X.shape
        counts = np.bincount(y)
        ent    = entropy(counts/counts.sum(), base=2)
        corr   = np.corrcoef(X, rowvar=False)
        iu     = np.triu_indices(n_f, k=1)
        avg_corr = np.nanmean(np.abs(corr[iu])) if iu[0].size else 0.0

        meta = {
            "dataset":         name,
            "n_instances":     n_i,
            "n_features":      n_f,
            "n_classes":       len(np.unique(y)),
            "feat_inst_ratio": n_f / n_i,
            "class_imbalance": float(counts.max() / counts.min()),
            "class_entropy":   float(ent),
            "mean_skew":       float(skew(X, axis=0).mean()),
            "mean_kurtosis":   float(kurtosis(X, axis=0).mean()),
            "avg_abs_corr":    float(avg_corr),
            "mean_variance":   float(X.var(axis=0).mean())
        }

        # split & scale
        X_tr, X_te, y_tr, y_te = train_test_split(
            X, y, test_size=0.2, stratify=y, random_state=0
        )
        X_tr = np.asarray(X_tr, dtype=np.float32)
        X_te = np.asarray(X_te, dtype=np.float32)
        scaler = StandardScaler().fit(X_tr)
        X_tr = scaler.transform(X_tr).astype(np.float32)
        X_te = scaler.transform(X_te).astype(np.float32)

        # one-hot labels (force int32 for indices, float32 for one-hot)
        y_tr_i  = pd.factorize(y_tr)[0].astype(np.int32)
        y_te_i  = pd.factorize(y_te)[0].astype(np.int32)
        y_tr_oh = to_categorical(y_tr_i).astype(np.float32)
        y_te_oh = to_categorical(y_te_i).astype(np.float32)

        # reshape for sequence models
        X_tr_s = X_tr.reshape(-1, n_f, 1).astype(np.float32)
        X_te_s = X_te.reshape(-1, n_f, 1).astype(np.float32)

        # Debug: print dtypes
        print("  X_tr_s dtype:", X_tr_s.dtype, "shape:", X_tr_s.shape)
        print("  y_tr_oh dtype:", y_tr_oh.dtype, "shape:", y_tr_oh.shape)

        for mname, builder in [("CNN", build_cnn),
                               ("RNN", build_rnn),
                               ("LSTM", build_lstm)]:
            print(f"   └─ {mname}… ", end="", flush=True)

            # clear any previous model / dtype state
            K.clear_session()

            model = builder(n_f, y_tr_oh.shape[1])

            history = model.fit(
                X_tr_s, y_tr_oh,
                validation_split=0.1,
                epochs=20,
                batch_size=32,
                verbose=0,
                callbacks=[EarlyStopping(patience=3, restore_best_weights=True)]
            )

            # print exactly how many epochs it ran
            epochs_ran = len(history.history["loss"])
            _, acc = model.evaluate(X_te_s, y_te_oh, verbose=0)

            print(f"stopped@{epochs_ran}, acc={acc:.3f}")

            rec = meta.copy()
            rec.update(model=mname, accuracy=float(acc))
            rows.append(rec)

    return pd.DataFrame(rows)

# ──────────────────────────────────────────────────────────────────────────────
# 4) TRAIN THE RANKER
# ──────────────────────────────────────────────────────────────────────────────
df_train = benchmark(train_ds)

feature_cols = [
    "n_instances", "n_features", "n_classes", "feat_inst_ratio",
    "class_imbalance", "class_entropy",
    "mean_skew", "mean_kurtosis",
    "avg_abs_corr", "mean_variance"
]
group_sizes = df_train.groupby("dataset").size().to_numpy()

ranker = LGBMRanker(
    objective="lambdarank",
    random_state=0,
    n_estimators=200
)
ranker.fit(df_train[feature_cols], df_train.accuracy, group=group_sizes)

# ──────────────────────────────────────────────────────────────────────────────
# 5) EVALUATE ON THE 10 HELD-OUT DATASETS
# ──────────────────────────────────────────────────────────────────────────────
df_test = benchmark(test_ds)
df_test["pred_score"] = ranker.predict(df_test[feature_cols])

for ds in df_test.dataset.unique():
    top3 = (
        df_test[df_test.dataset == ds]
               .sort_values("pred_score", ascending=False)
               .head(3)
    )
    print(f"\nTop-3 recommendations for {ds}:")
    print(top3[["model","accuracy","pred_score"]].to_string(index=False))

TensorFlow version: 2.18.0

→ [1/30] syn_32
  X_tr_s dtype: float32 shape: (785, 46, 1)
  y_tr_oh dtype: float32 shape: (785, 4)
   └─ CNN… stopped@9, acc=0.289
   └─ RNN… stopped@14, acc=0.102
   └─ LSTM… stopped@18, acc=0.183

→ [2/30] syn_17
  X_tr_s dtype: float32 shape: (504, 23, 1)
  y_tr_oh dtype: float32 shape: (504, 5)
   └─ CNN… stopped@8, acc=0.190
   └─ RNN… stopped@20, acc=0.230
   └─ LSTM… stopped@20, acc=0.270

→ [3/30] syn_30
  X_tr_s dtype: float32 shape: (904, 35, 1)
  y_tr_oh dtype: float32 shape: (904, 5)
   └─ CNN… stopped@8, acc=0.172
   └─ RNN… stopped@16, acc=0.075
   └─ LSTM… stopped@19, acc=0.167

→ [4/30] syn_36
  X_tr_s dtype: float32 shape: (1126, 45, 1)
  y_tr_oh dtype: float32 shape: (1126, 3)
   └─ CNN… stopped@4, acc=0.287
   └─ RNN… stopped@20, acc=0.387
   └─ LSTM… stopped@18, acc=0.277

→ [5/30] syn_33
  X_tr_s dtype: float32 shape: (782, 27, 1)
  y_tr_oh dtype: float32 shape: (782, 5)
   └─ CNN… stopped@7, acc=0.209
   └─ RNN… stopped@20, acc=0.184


LightGBMError: label should be int type (met 0.289340) for ranking task,
for the gain of label, please set the label_gain parameter

In [ ]:
print("df_test columns:", df_test.columns.tolist())
print("feature_cols:", feature_cols)

df_test columns: []
feature_cols: ['n_instances', 'n_features', 'n_classes', 'feat_inst_ratio', 'class_imbalance', 'class_entropy', 'mean_skew', 'mean_kurtosis', 'avg_abs_corr', 'mean_variance']


In [ ]:
# Re-create df_test with all meta-features and model results
df_test = benchmark(test_ds)

# Check that the expected columns are present
print("df_test columns:", df_test.columns.tolist())

# Predict scores with the trained ranker
df_test["pred_score"] = ranker.predict(df_test[feature_cols])

# Show top-3 recommendations for each test dataset
for ds in df_test.dataset.unique():
    top3 = (
        df_test[df_test.dataset == ds]
               .sort_values("pred_score", ascending=False)
               .head(3)
    )
    print(f"\nTop-3 recommendations for {ds}:")
    print(top3[["model", "accuracy", "pred_score"]].to_string(index=False))


→ [1/10] syn_2
  X_tr_s dtype: float32 shape: (639, 11, 1)
  y_tr_oh dtype: float32 shape: (639, 2)
   └─ CNN… stopped@20, acc=0.781
   └─ RNN… stopped@6, acc=0.944
   └─ LSTM… stopped@14, acc=0.944

→ [2/10] syn_31
  X_tr_s dtype: float32 shape: (751, 25, 1)
  y_tr_oh dtype: float32 shape: (751, 3)
   └─ CNN… stopped@20, acc=0.186
   └─ RNN… stopped@16, acc=0.319
   └─ LSTM… stopped@18, acc=0.319

→ [3/10] syn_3
  X_tr_s dtype: float32 shape: (411, 6, 1)
  y_tr_oh dtype: float32 shape: (411, 4)
   └─ CNN… stopped@20, acc=0.078
   └─ RNN… stopped@15, acc=0.029
   └─ LSTM… stopped@20, acc=0.029

→ [4/10] syn_21
  X_tr_s dtype: float32 shape: (1093, 25, 1)
  y_tr_oh dtype: float32 shape: (1093, 5)
   └─ CNN… stopped@5, acc=0.230
   └─ RNN… stopped@20, acc=0.029
   └─ LSTM… stopped@20, acc=0.102

→ [5/10] syn_27
  X_tr_s dtype: float32 shape: (893, 14, 1)
  y_tr_oh dtype: float32 shape: (893, 5)
   └─ CNN… stopped@7, acc=0.228
   └─ RNN… stopped@11, acc=0.348
   └─ LSTM… stopped@13, acc=

In [ ]:
# --- SAVE RESULTS AND RANKER ---

# Save the test results DataFrame
df_test.to_csv('df_test_results.csv', index=False)
print("Saved df_test to df_test_results.csv")

# Save the trained LightGBM ranker
import joblib
joblib.dump(ranker, 'lgbm_ranker.pkl')
print("Saved ranker to lgbm_ranker.pkl")